In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from bs4 import BeautifulSoup
import csv
import asyncio
import model
import tqdm as notebook_tqdm

c:\ProgramData\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup Selenium driver for access website without open new tab
chrome_options = Options()
chrome_options.add_argument("--headless") 
service = Service("./chromedriver.exe")
main_driver = webdriver.Chrome(service=service, options=chrome_options)
sub_driver = webdriver.Chrome(service=service, options=chrome_options)

ticker_symbol = "VCB"
start_date = "01/01/2022"  # Specify the start date
end_date = "31/12/2023"  # Specify the end date
url = f"https://s.cafef.vn/tin-doanh-nghiep/{ticker_symbol}/Event.chn"
base_url = "https://s.cafef.vn"
date_format = "%d/%m/%Y"
start_date = datetime.strptime(start_date, date_format).date()
end_date = datetime.strptime(end_date, date_format).date()
result = []

# Scrape the main page
main_driver.get(url)
# The current news list index, starting with 0
page_counter = 0
# Check if should the driver continue to click next button
is_while = True
# Define next button
next_button = main_driver.find_element(By.ID, "spanNext")

In [3]:
# May create a function to save data to csv file
class Article:
    def __init__(self, title, content, date):
        self.title = title
        self.content = content
        self.date = date
        self.sentiment = {"Negative": 0, "Positive": 0, "Neutral": 0} # Default level
    def __str__(self):
        return f"Title: {self.title}\nDate: {(self.date)}\nContent: {self.content}\nSentiment Percentages: {self.sentiment}\n"

In [4]:
# async def get_content(news_item):
#     news_link = news_item.find("a")["href"]
#     news_title = news_item.find("a").text.strip()
    


In [5]:
# May take very long time to scrape (~3 mins), time will depend on the speed of internet connection
while is_while:
    soup = BeautifulSoup(main_driver.page_source, "html.parser")
    news_list = soup.find("div", id="divEvents").find_all("li")
    
    for news_item in news_list:
        news_link = news_item.find("a")["href"]
        news_title = news_item.find("a").text.strip()
        created_date_time = news_item.find("span").text.strip()
        created_date = created_date_time.split(" ")[0]
        created_date = datetime.strptime(created_date, date_format).date()
        
        if (created_date > end_date):
            continue
        # All the li after are created before start_date
        elif (created_date < start_date):
            is_while = False
            break
        
        # Access article from the link
        sub_driver.get(base_url+news_link)
        
        new_soup = BeautifulSoup(sub_driver.page_source, "html.parser")
        news_div = new_soup.find("div", id="newscontent")
        news_content = news_div.get_text(strip=True)
        if (len(news_content) < 300):
            continue
        temp_article = Article(news_title, news_content, created_date)
        result.append(temp_article)

    # Scrape 30 page of item list only
    if (page_counter < 30):
        next_button.click()
        page_counter+=1
    else:    
        is_while=False
   

In [6]:
# Run this cell for perform sentiment analysis for all the article (may take 1-2 min)
for article in result:
    sentiment_rs = model.analyze_sentiment(article.title)
    article.sentiment["Negative"] = sentiment_rs[0]
    article.sentiment["Positive"] = sentiment_rs[1]
    article.sentiment["Neutral"] = sentiment_rs[2]
    

In [7]:
for article in result:
    print(article)

Title: Trở thành ngân hàng mẹ của Ngân hàng Xây dựng, Vietcombank được gì?
Date: 2023-06-06
Content: Chia sẻ tại buổi gặp gỡ báo chí ngày 5/6, ông Đàm Minh Đức, Thành viên Hội đồng thành viên kiêm Tổng giám đốc Ngân hàng Xây Dựng (CB), cho biết: "Dự kiến khoảng 6 tháng nữa Vietcombank sẽ trở thành ngân hàng mẹ của CB. Vừa qua, Bộ Chính trị cũng đã phê duyệt chủ trương này. CB cũng đang chuẩn bị sẵn sàng cho công tác chuyển giao bắt buộc về Vietcombank".Theo lãnh đạo CB, được chuyển giao về một trong những tổ chức tín dụng hàng đầu Việt Nam sẽ mở ra hành trình mới với CB, sau hơn 8 năm kiên định trên hành trình tái cơ cấu.Trên thị trường từ lâu đã đồn đoán về khả năng Vietcombank sẽ nhận chuyển giao CB. Trước đó, năm 2015, khi CB bị mua lại bắt buộc 0 đồng và kiểm soát đặc biệt do thua lỗ, âm vốn, Vietcombank đã được chỉ định tham gia hỗ trợ ngân hàng này tái cơ cấu.Về phíaVietcombank, năm 2022, ngân hàng đã xin ý kiến cổ đông về chủ trương nhận chuyển giao bắt buộc (CGBB) một tổ chức t

In [8]:
# Should quit the driver after execute all code
main_driver.quit()
sub_driver.quit()
